Import


In [3]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


création de la dataframe

In [4]:
df = pd.read_csv(r"After_Sort.csv", parse_dates=["BaseDateTime"])
df.sort_values("BaseDateTime", inplace=True)

print informations

In [5]:
print(df.head())
print(df.info())
print(df.describe())
print(df["VesselType"].value_counts())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO CallSign  VesselType  Status  \
117352    206.0       MERCURY SKY     IMO9796949   9V5148          70       0   
117395    294.0  PROVINCETOWN III     IMO9329394  WDB8185          60       0   
117339    184.0         COBIA LNG     IMO9869306  9HA5192          84       0   
117360      NaN   BELLE CHASSE II            NaN  WDD7069          60       0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  WDK3453          60       0   

        Length  Width  Draft  Cargo TransceiverClass  
1

ajout des colones lat et long +5, +10, +15 et delta_temps à la dataframe

In [6]:
nouvelles_colonnes = ['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15','Delta_temps']

# Ajout des colonnes avec des valeurs vides (NaN)
for col in nouvelles_colonnes:
    df[col] = None  # ou pd.NA, ou np.nan si vous utilisez numpy


print(df.head())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO  ... Draft  Cargo  \
117352    206.0       MERCURY SKY     IMO9796949  ...   NaN   70.0   
117395    294.0  PROVINCETOWN III     IMO9329394  ...   1.9   60.0   
117339    184.0         COBIA LNG     IMO9869306  ...   9.4   84.0   
117360      NaN   BELLE CHASSE II            NaN  ...   NaN   60.0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  ...   3.3   60.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \
117352                 A    None   None     Non

Remplissage de la colonne Delta_temps

In [7]:
df['Delta_temps'] = df.sort_values(['MMSI', 'BaseDateTime']).groupby('MMSI')['BaseDateTime'].diff().dt.total_seconds()

In [11]:
# Assurez-vous que le DataFrame est trié par MMSI et par temps, c'est crucial pour la suite.
df.sort_values(['MMSI', 'BaseDateTime'], inplace=True)

# Créez une copie du dataframe qui servira de table de recherche pour les positions futures.
df_lookup = df[['MMSI', 'BaseDateTime', 'LAT', 'LON']].copy()

# --- Fonction corrigée pour trouver les cibles pour un horizon donné ---
def find_future_positions(main_df, lookup_df, horizon_min, tolerance_sec):
    """
    Trouve la position future pour chaque ligne du dataframe principal en utilisant une méthode robuste.
    """
    # Créez une copie pour éviter de modifier les dataframes originaux
    df_temp = main_df.copy()
    
    # Calculez le temps cible pour la recherche
    df_temp['target_time'] = df_temp['BaseDateTime'] + pd.Timedelta(minutes=horizon_min)
    
    # Fusionnez pour trouver la position la plus proche dans le futur.
    # On utilise 'suffixes' pour gérer les colonnes LAT/LON qui existent dans les deux dataframes.
    # Les colonnes de 'lookup_df' (right) auront le suffixe '_future'.
    merged_df = pd.merge_asof(
        left=df_temp.sort_values('target_time'),
        right=lookup_df.sort_values('BaseDateTime'),
        left_on='target_time',
        right_on='BaseDateTime',
        by='MMSI',
        direction='nearest',
        tolerance=pd.Timedelta(seconds=tolerance_sec),
        suffixes=('', '_future') 
    )
    
    # Renommer les colonnes nouvellement ajoutées au format souhaité (ex: LAT_5, LONG_5)
    # C'est ici la correction majeure : on renomme APRES la fusion.
    rename_dict = {
        'LAT_future': f'LAT_{horizon_min}',
        'LON_future': f'LONG_{horizon_min}'
    }
    merged_df.rename(columns=rename_dict, inplace=True)
    
    # Rétablissez l'ordre initial du dataframe
    merged_df.sort_index(inplace=True)
    
    # Retournez juste les colonnes cibles
    return merged_df[[f'LAT_{horizon_min}', f'LONG_{horizon_min}']]

# --- Appliquez la fonction pour chaque horizon ---
targets_5 = find_future_positions(df, df_lookup, 5, 120)
targets_10 = find_future_positions(df, df_lookup, 10, 300)
targets_15 = find_future_positions(df, df_lookup, 15, 420)

# --- Concaténez les nouvelles colonnes au dataframe original ---
# On supprime d'abord les anciennes colonnes cibles si elles existent
df.drop(columns=['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15'], inplace=True, errors='ignore')

# On concatène les nouvelles cibles.
df = pd.concat([df, targets_5, targets_10, targets_15], axis=1)

# --- Vérification ---
print("Aperçu du résultat :")
print(df[['MMSI', 'BaseDateTime', 'LAT', 'LON', 'LAT_5', 'LONG_5', 'LAT_10', 'LONG_10', 'LAT_15', 'LONG_15']].head(10))

print("\nInformations sur le remplissage des colonnes cibles :")
print(df[['LAT_5', 'LONG_5', 'LAT_10', 'LONG_10', 'LAT_15', 'LONG_15']].info())

Aperçu du résultat :
             MMSI        BaseDateTime       LAT       LON  LAT_5     LAT_5  \
117823  205776000 2023-05-25 00:07:27  25.95847 -97.37876   None  28.42186   
118169  205776000 2023-05-25 00:13:27  25.95846 -97.37880   None  29.27150   
118391  205776000 2023-05-25 00:16:27  25.95847 -97.37880   None  28.49450   
125113  205776000 2023-05-25 00:31:27  25.95848 -97.37879   None  29.63941   
120283  205776000 2023-05-25 00:34:27  25.95846 -97.37882   None  29.34030   
119768  205776000 2023-05-25 00:37:27  25.95848 -97.37882   None  29.95513   
120279  205776000 2023-05-25 00:40:27  25.95846 -97.37883   None  29.10751   
121098  205776000 2023-05-25 00:43:27  25.95849 -97.37879   None  29.05616   
121336  205776000 2023-05-25 00:46:27  25.95846 -97.37878   None  28.34240   
120988  205776000 2023-05-25 00:55:27  25.95842 -97.37880   None  29.77716   

           LAT_5  LONG_5    LONG_5    LONG_5  ...    LAT_10 LONG_10   LONG_10  \
117823  29.25482    None -93.22272 -89.

Séparation en bases d'apprentissage, test et validation par MMSI

In [ ]:

# Récupération des MMSI uniques et suppr valeurs manquantes
mmsi_uniques = df['MMSI'].dropna().unique()

# Mélange aléatoire
np.random.seed(42)
np.random.shuffle(mmsi_uniques)

# Affectation fixe : 97 pour apprentissage, 30 pour test, 22 pour validation
n_train = 97
n_test = 30
n_val = 22

mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:n_train + n_test]
mmsi_val = mmsi_uniques[n_train + n_test:n_train + n_test + n_val]

# Création des trois sous-ensembles
df_train = df[df['MMSI'].isin(mmsi_train)]
df_test = df[df['MMSI'].isin(mmsi_test)]
df_val = df[df['MMSI'].isin(mmsi_val)]

# Affichage pour vérification
print("Nombre de MMSI uniques:")
print("Train:", df_train['MMSI'].nunique())
print("Test :", df_test['MMSI'].nunique())
print("Val  :", df_val['MMSI'].nunique())




Nombre de MMSI uniques:
Train: 97
Test : 30
Val  : 22


Observation de début de chaque df

In [ ]:
print(df_train.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_test.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_val.head(5))

In [ ]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head(10))


In [ ]:
print(f"Taille de l'ensemble d'apprentissage : {len(df_train)}")
print(f"Taille de l'ensemble de test : {len(df_test)}")
print(f"Taille de l'ensemble de validation : {len(df_val)}")


In [ ]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head())
# print(df_train.iloc[2950:3000])


prendre entre 295 et 305sec
prendre entre 595 et 605sec
prendre entre 895 et 905sec

In [ ]:
def interpoler_positions_dataframe(df):
    """
    Fonction pour interpoler les positions à 5, 10 et 15 minutes
    pour un dataframe donné
    """
    # Créer une copie pour éviter les warnings
    df_copy = df.copy()
    
    # Trier par MMSI et datetime pour assurer l'ordre chronologique
    df_copy = df_copy.sort_values(['MMSI', 'BaseDateTime']).reset_index(drop=True)
    
    print(f"Début de l'interpolation pour {len(df_copy)} lignes...")
    
    # Parcourir chaque navire
    for mmsi in df_copy['MMSI'].unique():
        # Filtrer les données du navire actuel
        mask_navire = df_copy['MMSI'] == mmsi
        indices_navire = df_copy[mask_navire].index.tolist()
        
        # Parcourir les positions de ce navire
        for i in range(len(indices_navire) - 1):
            idx_actuel = indices_navire[i]
            idx_suivant = indices_navire[i + 1]
            
            # Récupérer les positions et temps
            lat_A = df_copy.loc[idx_actuel, 'LAT']
            lon_A = df_copy.loc[idx_actuel, 'LON']
            time_A = df_copy.loc[idx_actuel, 'BaseDateTime']
            
            lat_B = df_copy.loc[idx_suivant, 'LAT']
            lon_B = df_copy.loc[idx_suivant, 'LON']
            time_B = df_copy.loc[idx_suivant, 'BaseDateTime']
            
            # Calculer l'intervalle de temps en secondes
            delta_temps = (time_B - time_A).total_seconds()
            
            # Interpolation à 5 minutes (300s)
            if delta_temps >= 300:
                fraction_5min = 300 / delta_temps
                lat_5min = lat_A + fraction_5min * (lat_B - lat_A)
                lon_5min = lon_A + fraction_5min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_5'] = lat_5min
                df_copy.loc[idx_actuel, 'LONG_5'] = lon_5min
            
            # Interpolation à 10 minutes (600s)
            if delta_temps >= 600:
                fraction_10min = 600 / delta_temps
                lat_10min = lat_A + fraction_10min * (lat_B - lat_A)
                lon_10min = lon_A + fraction_10min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_10'] = lat_10min
                df_copy.loc[idx_actuel, 'LONG_10'] = lon_10min
            
            # Interpolation à 15 minutes (900s)
            if delta_temps >= 900:
                fraction_15min = 900 / delta_temps
                lat_15min = lat_A + fraction_15min * (lat_B - lat_A)
                lon_15min = lon_A + fraction_15min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_15'] = lat_15min
                df_copy.loc[idx_actuel, 'LONG_15'] = lon_15min
    
    print("Interpolation terminée !")
    return df_copy

# Application de l'interpolation sur les trois ensembles
print("=== INTERPOLATION DE L'ENSEMBLE D'APPRENTISSAGE ===")
df_train_interpolated = interpoler_positions_dataframe(df_train)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE TEST ===")
df_test_interpolated = interpoler_positions_dataframe(df_test)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE VALIDATION ===")
df_val_interpolated = interpoler_positions_dataframe(df_val)

In [ ]:
def verifier_interpolation(df, nom_ensemble):
    """
    Fonction pour vérifier les résultats de l'interpolation
    """
    print(f"\n=== VÉRIFICATION - {nom_ensemble} ===")
    
    # Compter les valeurs non nulles pour chaque colonne d'interpolation
    interpolations_5min = df['LAT_5'].notna().sum()
    interpolations_10min = df['LAT_10'].notna().sum()
    interpolations_15min = df['LAT_15'].notna().sum()
    
    print(f"Interpolations à 5 minutes : {interpolations_5min}")
    print(f"Interpolations à 10 minutes : {interpolations_10min}")
    print(f"Interpolations à 15 minutes : {interpolations_15min}")
    
    # Afficher quelques exemples d'interpolation
    exemples = df[df['LAT_5'].notna()].head(3)
    if len(exemples) > 0:
        print(f"\nExemples d'interpolation à 5 minutes :")
        for idx, row in exemples.iterrows():
            print(f"MMSI {row['MMSI']} : Position originale ({row['LAT']:.5f}, {row['LON']:.5f}) -> Position à +5min ({row['LAT_5']:.5f}, {row['LONG_5']:.5f})")

# Vérification des trois ensembles
verifier_interpolation(df_train_interpolated, "TRAIN")
verifier_interpolation(df_test_interpolated, "TEST")
verifier_interpolation(df_val_interpolated, "VALIDATION")

print("\n=== INTERPOLATION TERMINÉE POUR TOUS LES ENSEMBLES ===")
print("Les DataFrames df_train_interpolated, df_test_interpolated et df_val_interpolated")
print("contiennent maintenant les positions interpolées dans les colonnes LAT_5, LONG_5, LAT_10, LONG_10, LAT_15, LONG_15")

In [ ]:
seuil_SOG = 0.5  # à ajuster selon ton dataset
df_filtre = df[df['SOG'] > seuil_SOG].reset_index(drop=True)
len(df_filtre)